In [1]:
import os
import warnings
import PIL
from PIL import Image
from typing import TYPE_CHECKING, Any, Callable, Dict, List, Optional, Tuple

import torch
import argparse
import whisper
import numpy as np

from gradio import processing_utils
from modeling.BaseModel import BaseModel
from modeling import build_model
from utils.distributed import init_distributed
from utils.arguments import load_opt_from_config_files
from utils.constants import COCO_PANOPTIC_CLASSES

from demo.seem.tasks import *

conf_files = "configs/seem/focall_unicl_lang_demo.yaml"

opt = load_opt_from_config_files([conf_files])
opt = init_distributed(opt)

Deformable Transformer Encoder is not available.


In [2]:
cur_model = 'None'
pretrained_pth = '~/autodl-tmp'

if 'focalt' in conf_files:
    pretrained_pth = os.path.join(pretrained_pth, "seem_focalt_v0.pt")
    cur_model = 'Focal-T'
elif 'focall' in conf_files:
    pretrained_pth = os.path.join(pretrained_pth, "seem_focall_v0.pt")
    cur_model = 'Focal-L'
    # this

model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()

with torch.no_grad():
    model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(COCO_PANOPTIC_CLASSES + ["background"], is_eval=True)


'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /openai/clip-vit-base-patch32/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fab40da5190>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 2f286127-2146-4f92-a859-3b9a9fe91c9c)')' thrown while requesting HEAD https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/tokenizer_config.json
'(MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /openai/clip-vit-base-patch32/resolve/main/vocab.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x7fab40da57c0>, 'Connection to huggingface.co timed out. (connect timeout=10)'))"), '(Request ID: 21d3190b-e82f-49b1-917d-11edecc3bd05)')' thrown while requesting HEAD https://huggingface.co/openai/clip-vit-base-patch32/resolve/main/vocab.json
'(MaxRetryError("HTTPS

OSError: Can't load tokenizer for 'openai/clip-vit-base-patch32'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'openai/clip-vit-base-patch32' is the correct path to a directory containing all relevant files for a CLIPTokenizer tokenizer.

In [ ]:
audio = whisper.load_model("base")

@torch.no_grad()
def inference(image, task, *args, **kwargs):
    with torch.autocast(device_type='cuda', dtype=torch.float16):
        if 'Video' in task:
            return interactive_infer_video(model, audio, image, task, *args, **kwargs)
        else:
            return interactive_infer_image(model, audio, image, task, *args, **kwargs)


In [ ]:
from PIL import Image
input_image = Image.open('../2.jpg')


In [ ]:
image = interactive_infer_image(model, audio, input_image, ['Task'])
print(image)